# Calculating landcover distribution & vegetation extraction
**Farzad Vahidi Mayamey**	

**Aim of the studies**
1. Calculating landcover distribution in our wetlands based on 10 m landcover map of Sweden
2. Open water and flooded vegetation extraction in one of our wetlands.

## Calculating landcover distribution
we use two different layer:
1. [National landcover map of Sweden in 10m resolution](https://www.swedishepa.se/State-of-the-environment/Maps-and-map-services/National-Land-Cover-Database/) (above picture)
2. [vector layer of Ramsar sites in Sweden](https://rsis.ramsar.org/ris-search/?f[0]=regionCountry_en_ss%3ASweden) (below picture)

<img src='Farzad_VahidiMayamey_sw2021_1.png' width="400">
\\
<img src='Farzad_VahidiMayamey_sw2021_2.png' width="400">

As NoData Value is not assigned, we assign nodata value = 0

we create the shapefile of each wetland by using ```$ split vector layer``` tool in QGIS to clip our landcover by this layers

Now that we have our clipped land cover we can calculate the histogram (in percentage)

Now we have the text file of our histograms we can extract our desired classes for each of our 67 wetlands
- we have 25 classess in total
- 7 of the classess are our main interest

<img src='Farzad_VahidiMayamey_sw2021_3.png' width="500"> 

Extract all 25 classes

Extract 7 main classess of our interest

In [1]:
%%bash
for file in hist/*.txt; do echo $file; filename=$(basename $file .txt);
    awk '$1==111 || $1==112 || $1==113 || $1==114 ||$1==115 || $1==116 ||$1==117 || $1==118 {sum1+= $2} 
$1==121 || $1==122 || $1==123 || $1==124 ||$1==125 || $1==126 ||$1==127 || $1==128 {sum2+= $2} 
$1==2 {sum3+= $2} 
$1==3 {sum4+= $2} 
$1==41 || $1==42 {sum5+= $2} 
$1==51 || $1==52 || $1==53 {sum6+= $2} 
$1==61 || $1==62 {sum7+= $2} 
END{print ("1.1 "sum1); print ("1.2 "sum2); print("2 "sum3); print("3 "sum4); print("4 "sum5); print("5 "sum6); print("6 "sum7)}' $file > class_7/${filename}.txt;done

hist/*.txt


bash: line 9: class_7/*.txt: No such file or directory


CalledProcessError: Command 'b'for file in hist/*.txt; do echo $file; filename=$(basename $file .txt);\n    awk \'$1==111 || $1==112 || $1==113 || $1==114 ||$1==115 || $1==116 ||$1==117 || $1==118 {sum1+= $2} \n$1==121 || $1==122 || $1==123 || $1==124 ||$1==125 || $1==126 ||$1==127 || $1==128 {sum2+= $2} \n$1==2 {sum3+= $2} \n$1==3 {sum4+= $2} \n$1==41 || $1==42 {sum5+= $2} \n$1==51 || $1==52 || $1==53 {sum6+= $2} \n$1==61 || $1==62 {sum7+= $2} \nEND{print ("1.1 "sum1); print ("1.2 "sum2); print("2 "sum3); print("3 "sum4); print("4 "sum5); print("5 "sum6); print("6 "sum7)}\' $file > class_7/${filename}.txt;done\n'' returned non-zero exit status 1.

Now we want to add header to each of the text files 

In [2]:
%%bash
for file in class_7/*.txt; do echo $file; filename=$(basename $file .txt);
awk 'BEGIN{print "Class Ratio"}1' $file > head/${filename}.txt; done #The 1 is to indicate to print every line of the file.

class_7/*.txt


bash: line 2: head/*.txt: No such file or directory


CalledProcessError: Command 'b'for file in class_7/*.txt; do echo $file; filename=$(basename $file .txt);\nawk \'BEGIN{print "Class Ratio"}1\' $file > head/${filename}.txt; done #The 1 is to indicate to print every line of the file.\n'' returned non-zero exit status 1.

In [3]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [4]:
import matplotlib.pyplot as plt
import os
import glob

file_location = os.path.join('class_7', '*.txt')
filenames = glob.glob(file_location)


class_ = ['Forest not on wetland', 'Forest on wetland',
          'Open wetland','Arable land', 'Other openland', 'Artificial surfaces',
          'Water']

for f in filenames:
    ratio = []
    
    outfile = open(f,'r')
    for line in outfile:
    	Data = [i for i in line.split()]
    	New_Data= [ j for j in Data[1].split()]
    	ratio.append(New_Data[0])
    outfile.close()
    
    file_name = os.path.basename(f)
    file_name=os.path.splitext(file_name)[0]
    print(file_name)
    
    ratio = [float(i) for i in ratio]
    #print(ratio)
    colors = ['green', 'lime', 'mediumpurple', 'khaki','orange', 'red', 'blue']
    explode = [0, 0.01, 0.01, 0, 0, 0, 0.01]
    # plotting pie chart
    def my_autopct(pct):
        return ('%.2f%%' % pct) if pct > 5 else ''
    
    plt.pie(ratio, colors = colors, startangle = 90, explode = explode,
    		 shadow = False, autopct=my_autopct )
    
    plt.title(label=file_name,  loc='center')
    ratio_r =  [round(num, 2) for num in ratio]
    labels = ['{0} - {1} %'.format(i, j) for i, j in zip(class_,ratio_r)]
    plt.legend( labels, loc='lower left', bbox_to_anchor=(-0.15, -.15), fontsize=7)
    plt.axis('equal')
    #plt.tight_layout()
    file_name_ext=os.path.splitext(file_name)[0]+".png"
    plt.savefig(file_name_ext, format="png", dpi=300)
    plt.show()
    


## Open water and flooded vegetation extraction


### Open water extraction
- Layers: SAR intensity sentinel-1 image. 

<img src='Farzad_VahidiMayamey_sw2021_4.png' width="400"> 

data source: [Google Earth Engine](https://code.earthengine.google.com)
         
          
we use thresholding method to extract open water from S1 layer 

First we move all layers to the crs: EPSG:4326

In [5]:
%%bash
rm --force layers_reprj/*.tif
for file in layers/*.tif; do echo $file; filename=$(basename $file .tif);
gdalwarp -ot "Float32" -t_srs EPSG:4326 $file layers_reprj/${filename}_reprj.tif;done

layers/*.tif


ERROR 4: layers/*.tif: No such file or directory


CalledProcessError: Command 'b'rm --force layers_reprj/*.tif\nfor file in layers/*.tif; do echo $file; filename=$(basename $file .tif);\ngdalwarp -ot "Float32" -t_srs EPSG:4326 $file layers_reprj/${filename}_reprj.tif;done\n'' returned non-zero exit status 2.

In [6]:
%%bash 
#applying nodata value equal to zero
for file in layers/*.tif; do echo $file; 
gdal_edit.py -a_nodata -3.39999995214436425e+38 $file;done

layers/*.tif


CalledProcessError: Command 'b'#applying nodata value equal to zero\nfor file in layers/*.tif; do echo $file; \ngdal_edit.py -a_nodata -3.39999995214436425e+38 $file;done\n'' returned non-zero exit status 255.

In [7]:
%%bash
rm --force crop/*.tif
for file in layers/*.tif; do echo $file; 
  filename=$(basename $file .tif)
  gdalwarp -overwrite -te 14.6714649 58.2914991 14.9516494 58.4129513 -tr 0.000269494585236 -0.000269494585236 $file crop/${filename}_crop.tif 
done

layers/*.tif


ERROR 4: layers/*.tif: No such file or directory


CalledProcessError: Command 'b'rm --force crop/*.tif\nfor file in layers/*.tif; do echo $file; \n  filename=$(basename $file .tif)\n  gdalwarp -overwrite -te 14.6714649 58.2914991 14.9516494 58.4129513 -tr 0.000269494585236 -0.000269494585236 $file crop/${filename}_crop.tif \ndone\n'' returned non-zero exit status 2.

we maskout values greater than **-23** 

<img src='Farzad_VahidiMayamey_sw2021_5.png' width="400"> 

In [8]:
%%bash
pkgetmask -max -23 -data 2 -nodata 0 -ot Byte -i crop/SAR_VH_crop.tif -o crop/water.tif;
gdalwarp -crop_to_cutline -cutline rms/ramsarid_23.gpkg crop/water.tif  crop/wat.tif; 

0

terminate called after throwing an instance of 'std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >'
bash: line 1: 235566 Aborted                 (core dumped) pkgetmask -max -23 -data 2 -nodata 0 -ot Byte -i crop/SAR_VH_crop.tif -o crop/water.tif
ERROR 4: crop/water.tif: No such file or directory


CalledProcessError: Command 'b'pkgetmask -max -23 -data 2 -nodata 0 -ot Byte -i crop/SAR_VH_crop.tif -o crop/water.tif;\ngdalwarp -crop_to_cutline -cutline rms/ramsarid_23.gpkg crop/water.tif  crop/wat.tif; \n'' returned non-zero exit status 2.

### Flooded vegetation extraction by using unsupervised Kmean classification
- Layers: 1. Coherence layer ( left picture)
          2. slope layer (generated from SRTM DEM (right picture)

<img src='Farzad_VahidiMayamey_sw2021_6.png' width="400">  <img src='Farzad_VahidiMayamey_sw2021_7.png' width="400"> 

orfeo kmean clustering toolbox could not handle **nan** values and alsso give us warning for **nodatavalue=-3.39999995214436425e+38** 
by the following code we solve this two problems

In [9]:
%%bash
#crop the coherence, SAR, and slope .tif files by our wetland shape of interest to exclude the nan values
gdalwarp -crop_to_cutline -overwrite -srcnodata -3.39999995214436425e+38 -dstnodata -999 -cutline rms/ramsarid_23.gpkg crop/slope_crop.tif  crop/slope_crop_ready.tif; 
gdalwarp -crop_to_cutline -overwrite -srcnodata -3.39999995214436425e+38 -dstnodata -999 -cutline rms/ramsarid_23.gpkg crop/coherence_crop.tif  crop/coherence_crop_ready.tif 

ERROR 4: crop/slope_crop.tif: No such file or directory
ERROR 4: crop/coherence_crop.tif: No such file or directory


CalledProcessError: Command 'b'#crop the coherence, SAR, and slope .tif files by our wetland shape of interest to exclude the nan values\ngdalwarp -crop_to_cutline -overwrite -srcnodata -3.39999995214436425e+38 -dstnodata -999 -cutline rms/ramsarid_23.gpkg crop/slope_crop.tif  crop/slope_crop_ready.tif; \ngdalwarp -crop_to_cutline -overwrite -srcnodata -3.39999995214436425e+38 -dstnodata -999 -cutline rms/ramsarid_23.gpkg crop/coherence_crop.tif  crop/coherence_crop_ready.tif \n'' returned non-zero exit status 2.

We want to build stack of 2 layers before doing unsupervised classification: **SAR coherence** and **SRTM DEM**

In [10]:
%%bash
# stck the tif
gdalbuildvrt -overwrite -separate crop/stack.vrt crop/coherence_crop_ready.tif crop/slope_crop_ready.tif
gdalinfo -mm crop/stack.vrt

0...10...20...30...40...50...60...70...80...90...100 - done.


ERROR 4: crop/coherence_crop_ready.tif: No such file or directory
Warning 1: Can't open crop/coherence_crop_ready.tif. Skipping it
ERROR 4: crop/slope_crop_ready.tif: No such file or directory
Warning 1: Can't open crop/slope_crop_ready.tif. Skipping it
ERROR 4: crop/stack.vrt: No such file or directory
gdalinfo failed - unable to open 'crop/stack.vrt'.


CalledProcessError: Command 'b'# stck the tif\ngdalbuildvrt -overwrite -separate crop/stack.vrt crop/coherence_crop_ready.tif crop/slope_crop_ready.tif\ngdalinfo -mm crop/stack.vrt\n'' returned non-zero exit status 1.

In [11]:
%%bash
# unsupervised classification
~/OTB/superbuild_install/bin/otbcli_KMeansClassification -in crop/stack.vrt -ts 1000 -nc 15 -maxit 1000 -out crop/ClassificationFilterOutput.tif uint8

bash: line 2: /home/selv/OTB/superbuild_install/bin/otbcli_KMeansClassification: No such file or directory


CalledProcessError: Command 'b'# unsupervised classification\n~/OTB/superbuild_install/bin/otbcli_KMeansClassification -in crop/stack.vrt -ts 1000 -nc 15 -maxit 1000 -out crop/ClassificationFilterOutput.tif uint8\n'' returned non-zero exit status 127.

In the following code we select the cluster with highest coherence **(cluster n. 12)**

<img src='Farzad_VahidiMayamey_sw2021_8.png' width="400">  <img src='Farzad_VahidiMayamey_sw2021_9.png' width="400"> 

In [12]:
%%bash
pkgetmask -max 12 -min 12 -data 1 -nodata 0 -ot Byte -i crop/ClassificationFilterOutput.tif -o crop/veg.tif

0

terminate called after throwing an instance of 'std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >'
bash: line 1: 235624 Aborted                 (core dumped) pkgetmask -max 12 -min 12 -data 1 -nodata 0 -ot Byte -i crop/ClassificationFilterOutput.tif -o crop/veg.tif


CalledProcessError: Command 'b'pkgetmask -max 12 -min 12 -data 1 -nodata 0 -ot Byte -i crop/ClassificationFilterOutput.tif -o crop/veg.tif\n'' returned non-zero exit status 134.

Now we want to calculate number of water pixels and flooded Vegetation

In [13]:
%%bash
#convert .tif to .txt
gdal_translate -of XYZ crop/wat.tif crop/wat.txt
gdal_translate -of XYZ crop/veg.tif crop/veg.txt


ERROR 4: crop/wat.tif: No such file or directory
ERROR 4: crop/veg.tif: No such file or directory


CalledProcessError: Command 'b'#convert .tif to .txt\ngdal_translate -of XYZ crop/wat.tif crop/wat.txt\ngdal_translate -of XYZ crop/veg.tif crop/veg.txt\n'' returned non-zero exit status 1.

In [14]:
%%bash
#calculating number of pixels
awk '{print$3}' crop/wat.txt | grep 2 | wc -l > crop/final.txt;
awk '{print$3}' crop/veg.txt | grep 1 | wc -l >> crop/final.txt;
cat crop/final.txt

bash: line 2: crop/final.txt: No such file or directory
awk: fatal: cannot open file `crop/wat.txt' for reading (No such file or directory)
bash: line 3: crop/final.txt: No such file or directory
awk: fatal: cannot open file `crop/veg.txt' for reading (No such file or directory)
cat: crop/final.txt: No such file or directory


CalledProcessError: Command 'b"#calculating number of pixels\nawk '{print$3}' crop/wat.txt | grep 2 | wc -l > crop/final.txt;\nawk '{print$3}' crop/veg.txt | grep 1 | wc -l >> crop/final.txt;\ncat crop/final.txt\n"' returned non-zero exit status 1.

In [15]:
!jupyter nbconvert my_project.ipynb --to html

[NbConvertApp] WARNING | pattern 'my_project.ipynb' matched no files
This application is used to convert notebook files (*.ipynb) to various other
formats.


Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--debug
    set log level to logging.DEBUG (maximize logging output)
--generate-config
    generate default config file
-y
    Answer yes to any questions instead of prompting.
--execute
    Execute the notebook prior to export.
--allow-errors
    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.
--stdin
    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'
--stdout
    Write notebook output to stdout